In [9]:
import numpy as np
import matplotlib.pyplot as plt
import vegas
from geometry import get_rprime,get_d,get_D,get_cosPsi
from crossSection import get_BDMv,get_BDMp,diffCrossSectionNuDM,get_Ev_cosPhi
from constants import *

In [10]:
def dnv(d,Ev,Lv = Lv,tau = 10):
    """
    Neutrino number density per energy at d
    
    Input
    ------
    Enu: Neutrino energy in MeV
    d: Distance to the SN explosion site, in kpc
    Lv: Neutrino luminosity, default is 1e52 erg/s
    tau: duration of the SN explosion
    
    Output
    ------
    Neutrino flux at d: # per Enu per cm**3
    """
    Lv = Lv*erg2MeV*tau
    r = r*kpc2cm
    
    def _fv(Ev,Tv):
        """
        Fermi-Dirac distribution
        """
        exponent = Ev/Tv - 3
        # setup a cutoff value when the exponent beyon the validatiy of float64
        if exponent <= 709.782:
            return (1/18.9686)*Tv**(-3)*(Ev**2/(np.exp(exponent)+1))
        else:
            return 0
    
    nue_dist = _fv(Ev,2.76)/11
    nueb_dist = _fv(Ev,4.01)/16
    # total 4 species for x
    nux_dist = _fv(Ev,6.26)/25
    
    L=Lv/(4*np.pi*r**2*c)
    return L*(nue_dist+nueb_dist+4*nux_dist)


def nx(r,mx,kind='MW'):
    """
    DM number density at r
    
    Input
    ------
    r: distance to GC, in kpc
    mx: DM mass in MeV
    kind: string, MW or LMC
    
    Output
    ------
    DM number density, #/cm^3 at r
    """
    if kind == 'MW':
        rho,rs,n = 184,24.42,2
    elif kind == 'LMC':
        rho,rs,n = 68,31.9,3
    rr=r/rs
    return (rho/mx)/(rr*(1+rr)**n)

The term $dE_\nu/dT_\chi$ can be evaluated through the reciprocal of
$$
\frac{dT_\chi}{dE_\nu} = \frac{d}{dE_\nu}\left( E_\nu - \frac{E_\nu m_\chi}{m_\chi + E_\nu(1-\cos\phi)}\right)
$$
where
$$
\cos\phi = \frac{1}{\sqrt{1+\frac{\sin^2\psi}{(E_\nu/p_\chi -\cos\psi)^2}}}.
$$
We solved this with 
$$
\frac{d T_\chi}{dE_\nu}=1+\frac{E_{\nu}m_{\chi}}{[m_{\chi}+E_{\nu}(1-1/\sqrt{\xi})]^{2}}\left[1-\frac{E_{\nu}\sin^{2}\psi}{p_{\chi}(\frac{E_{\nu}}{p_{\chi}}-\cos\psi)^{3}\xi^{3/2}}-\frac{1}{\sqrt{\xi}}\right]-\frac{m_{\chi}}{m_{\chi}+E_{\nu}(1-1/\sqrt{\xi})}
$$
where
$$
\xi = 1+\frac{\sin^2\psi}{(E_\nu/p_\chi -\cos\psi)^2}
$$

In [12]:
def dEvdTx(Ev,Tx,mx,psi):
    px = get_BDMp(Tx,mx)
    xi = 1 + np.sin(psi)**2/(Ev/px - np.cos(psi))**2
    A = Ev*mx/(mx + Ev*(1 - 1/np.sqrt(xi)))**2
    B = 1 - Ev*np.sin(psi)**2/px/(Ev/px - np.cos(psi))**3/xi**1.5 - 1/np.sqrt(xi)
    C = mx/(mx + Ev*(1 - 1/np.sqrt(xi)))
    dTxdEv = 1 + A*B - C
    return 1/dTxdEv


def xemissivity(Tx,mx,r,d,psi,max_Ev = 100):
    
    diffCrossSectionNuDM(psi, Tx, mx, mV, gV, gD, max_Ev = max_Ev)

In [15]:
diffCrossSectionNuDM?

Signature: diffCrossSectionNuDM(psi, Tx, mx, mV, gV, gD, max_Ev=2000)
Docstring:
Lab-frame differential cross section for neutrino-DM scattering

Input
------
psi: the DM scattering angle in rad
Tx: the BDM kinetic energy in MeV
mx: DM mass in MeV
mV: mediator mass in MeV
gV: the nu-DM coupling constant
gD: the DM-DM coupling constant
max_Ev: maximum Ev to be searched for the solution of the algorithm

Output
------
nu-DM diff. cross section: Lab-frame with unit 1/cm^2 * 1/rad
File:      ~/GitHub/SNnuBDM2/src/crossSection.py
Type:      function


In [14]:
get_BDMp(1,0.01)

1.0099504938362078